In [1]:
import ctypes
import time
import os
import multiprocessing

In [2]:
_CCnt = ctypes.CDLL('./CCnt.so')

In [3]:
#initialize counter on both CPUs

# Launch process on CPU0
p1 = multiprocessing.Process(target=_CCnt.init, args=(1,0))
os.system("taskset -p -c {} {}".format(0, p1.pid))
p1.start() # start the process

# Launch process on CPU1
p2 = multiprocessing.Process(target=_CCnt.init, args=(1,0))
os.system("taskset -p -c {} {}".format(1, p2.pid))
p2.start() # start the process

p1.join()
p2.join()
print('CPU Counters initialized')

CPU Counters initialized


In [4]:
signed = _CCnt.getCC()
unsigned = signed
if unsigned < 0:
    unsigned += 2**32
print(signed)
print(unsigned)

764953586
764953586


In [5]:
t1 = 4294967295
t2 = 1
t3 = t2-t1
t3 += 2**32
print(t3)

2


In [6]:
def recur_fibo(n):
   if n <= 1:
       return n
   else:
       return(recur_fibo(n-1) + recur_fibo(n-2))


In [12]:
def timed_fib(n):
    t_before = time.time()
    cc_before = _CCnt.getCC()

    recur_fibo(n)
        
    cc_after = _CCnt.getCC()
    t_after = time.time()

    #convert to unsigned
    if cc_after < 0:
        cc_after += 2**32
    if cc_before < 0:
        cc_before += 2**32
        
    elapsed_time = t_after-t_before
    num_cycles = cc_after-cc_before
    
    #account for possibility of overflow
    if num_cycles < 0:
        num_cycles += 2**32
        
    print(elapsed_time)
    print(num_cycles)

In [19]:
n = 30
CPU = 1
p_fib = multiprocessing.Process(target=timed_fib, args=(n,))
os.system("taskset -p -c {} {}".format(CPU, p_fib.pid))
p_fib.start()


3.6484627723693848
2371319662


In [91]:
timed_fib(20)

0.059044599533081055
29599485
